# 🐧 This is the first `LangGraph` Agentic AI application. 

### 🚶‍➡️Here are the following steps.
1. Define the state class
2. Start the Graph bulider
3. Define the node
4. Define the edges
5. Compile the graph

- `STATE` :  represents current snapshot of the application
    - State is immutable
- `NODE`  :  Python code that do the work. Take state as input and produce new state
- `EDGE`  :  Edge choose what to do next


In [ ]:
from typing import Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from pydantic import BaseModel
import random
import os


In [ ]:
load_dotenv(override=True)

In [ ]:
MODEL_NAME = os.getenv(key="MODEL_NAME")


In [ ]:
# We can choose any model to perform this operation. I am choosing a open model here ollama. Both the code are present 
# over here.

# Initialize Ollama instead of ChatOpenAI
llm = ChatOllama(
    model="llama3.2",
    temperature=0,
    # base_url="http://localhost:11434"  # Default value, usually not needed
)

# Open AI Code. I commenting this code for now to use Ollama
# llm = ChatOpenAI(model=MODEL_NAME)

🚶‍➡️Step 1: Define the State class
- State can a Pydentic BaseModel or TypedDict

In [ ]:
class State(BaseModel):        
    messages: Annotated[list, add_messages]

🚶‍➡️Step 2: Start the Graph Builder with the State Class

In [ ]:
graph_builder = StateGraph(State)

🚶‍➡️Step 3: Define the nodes

In [ ]:
def basic_chat_node(old_state: State) -> State:

    response = llm.invoke(old_state.messages)
    new_state = State(messages=[response])
    return new_state

graph_builder.add_node("basic_chat", basic_chat_node)

🚶‍➡️Step 4: Setup the edges

In [ ]:
graph_builder.add_edge(START, "basic_chat")
graph_builder.add_edge("basic_chat", END)

🚶‍➡️Step 5: Compile the edges

In [ ]:
graph = graph_builder.compile()

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
def chat(user_input: str, history):
    message = {"role": "user", "content": user_input}
    messages = [message]
    state = State(messages=messages)
    result = graph.invoke(state)
    print(result)
    return result["messages"][-1].content


gr.ChatInterface(chat).launch()